# Лекция 6: Фундаментални модели и данни за предварително обучение

**Продължителност:** 2-2.5 часа  
**Предпоставки:** Лекция 5 (Transformer архитектура)  
**Следваща лекция:** Емерджентни способности при мащаб

---
## Цели на лекцията

След тази лекция ще можете:

- Обяснявате разликата между MLM и autoregressive езикови модели
- Разбирате какви данни се използват за обучение на LLM
- Имплементирате качествени филтри и дедупликация (MinHash)
- Прилагате scaling laws за предвиждане на производителност
- Идентифицирате контаминация в данни и бенчмаркове

### Пътна карта

```
1. Мотивация → 2. Pretraining Objectives → 3. Източници на данни
       ↓
4. Качество и филтриране → 5. Контаминация → 6. Scaling Laws
       ↓
7. Обучение в мащаб → 8. От base към useful модел → 9. Обобщение
```

In [ ]:
# Основни библиотеки
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import hashlib
import re

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# Настройки
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

# Възпроизводимост
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")
print("Библиотеките са заредени успешно.")

---
## 1. Мотивация: Парадигмата на фундаменталните модели

### От специализирани към универсални модели

| Период | Подход | Пример |
|--------|--------|--------|
| Преди 2018 | Task-specific модели | Отделен модел за всяка задача |
| 2018-2020 | Pretrain + finetune | BERT за различни NLP задачи |
| 2020+ | Foundation models | Един GPT за всичко |

### Трите съставки на Foundation Models

```
Foundation Model = Архитектура + Pretraining + Мащаб
                   (Transformer)   (Next token)   (Трилиони токени)
```

**Тази лекция:** Фокус върху **Pretraining** и **Данните**

### Self-Supervised Learning: Безплатни етикети

**Традиционно:** Нужни са човешки етикети (скъпо, бавно)

**Self-supervised:** Самият текст съдържа "етикети"

- **MLM:** Скрий дума → предвиди я
- **Autoregressive:** Дадени първи N думи → предвиди N+1

Така можем да използваме **целия интернет** за обучение!

In [ ]:
# Демонстрация: Self-supervised "етикети" са вградени в текста
text = "Котката седи на дивана."

# Autoregressive: всяка позиция е пример за обучение
tokens = text.split()
print("Autoregressive training examples:")
print("="*50)
for i in range(1, len(tokens)):
    context = " ".join(tokens[:i])
    target = tokens[i]
    print(f"Input: '{context}' → Target: '{target}'")

print("\n" + "="*50)
print(f"От 1 изречение получаваме {len(tokens)-1} training примера!")

### Един модел, много приложения

Foundation model може да:

- Отговаря на въпроси
- Превежда текст
- Пише код
- Обобщава документи
- Анализира sentiment

**Без специално обучение за всяка задача!**

---
## 2. Pretraining Objectives: Как учим от текст

### Два основни подхода

| Подход | Модели | Идея |
|--------|--------|------|
| **Masked LM** | BERT, RoBERTa | Скрий токени, предвиди ги |
| **Autoregressive LM** | GPT, LLaMA | Предвиди следващия токен |

### 2.1 Masked Language Modeling (MLM)

**BERT-style подход:**

1. Вземи изречение
2. Замаскирай 15% от токените с [MASK]
3. Предвиди маскираните токени

$$\mathcal{L}_{MLM} = -\sum_{i \in M} \log P(x_i | x_{\backslash M})$$

Където $M$ е множеството от маскирани позиции.

In [ ]:
# MLM демонстрация
def demonstrate_mlm(sentence, mask_prob=0.15):
    """Показва как MLM маскира и предвижда токени."""
    tokens = sentence.split()
    masked_tokens = tokens.copy()
    masked_positions = []
    
    # Маскираме ~15% от токените
    np.random.seed(42)
    for i in range(len(tokens)):
        if np.random.random() < mask_prob:
            masked_positions.append(i)
            masked_tokens[i] = "[MASK]"
    
    print(f"Original:  {sentence}")
    print(f"Masked:    {' '.join(masked_tokens)}")
    print(f"\nМоделът трябва да предвиди:")
    for pos in masked_positions:
        print(f"  Position {pos}: '{tokens[pos]}'")

sentence = "Големият черен котарак спеше спокойно на топлия диван в хола"
demonstrate_mlm(sentence, mask_prob=0.2)

### MLM: Предимства и недостатъци

| Предимства | Недостатъци |
|------------|-------------|
| Bidirectional context | Не може да генерира естествено |
| Добър за "разбиране" | [MASK] не съществува при inference |
| Ефективно за classification | Ограничен до understanding tasks |

In [ ]:
# Визуализация: MLM вижда в двете посоки
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

tokens_viz = ['The', 'cat', '[MASK]', 'on', 'mat']
n = len(tokens_viz)

# MLM - bidirectional
ax = axes[0]
for i, tok in enumerate(tokens_viz):
    color = 'red' if tok == '[MASK]' else 'steelblue'
    ax.add_patch(plt.Rectangle((i*1.5, 0), 1, 0.8, fill=True, color=color, alpha=0.7))
    ax.text(i*1.5 + 0.5, 0.4, tok, ha='center', va='center', fontsize=10, 
            color='white', fontweight='bold')

# Стрелки от всички към [MASK]
mask_idx = 2
for i in range(n):
    if i != mask_idx:
        ax.annotate('', xy=(mask_idx*1.5 + 0.5, 0.9), xytext=(i*1.5 + 0.5, 0.9),
                    arrowprops=dict(arrowstyle='->', color='green', alpha=0.6,
                                   connectionstyle='arc3,rad=0.3'))

ax.set_xlim(-0.5, n*1.5)
ax.set_ylim(-0.5, 2)
ax.axis('off')
ax.set_title('MLM: Bidirectional\n[MASK] вижда минало И бъдеще', fontsize=12)

# Autoregressive - causal
tokens_ar = ['The', 'cat', 'sat', 'on', '?']
ax = axes[1]
for i, tok in enumerate(tokens_ar):
    color = 'orange' if tok == '?' else 'steelblue'
    ax.add_patch(plt.Rectangle((i*1.5, 0), 1, 0.8, fill=True, color=color, alpha=0.7))
    ax.text(i*1.5 + 0.5, 0.4, tok, ha='center', va='center', fontsize=10,
            color='white', fontweight='bold')

# Стрелки само от миналото
target_idx = 4
for i in range(target_idx):
    ax.annotate('', xy=(target_idx*1.5 + 0.5, 0.9), xytext=(i*1.5 + 0.5, 0.9),
                arrowprops=dict(arrowstyle='->', color='green', alpha=0.6,
                               connectionstyle='arc3,rad=0.3'))

ax.set_xlim(-0.5, n*1.5)
ax.set_ylim(-0.5, 2)
ax.axis('off')
ax.set_title('Autoregressive: Causal\n? вижда само миналото', fontsize=12)

plt.tight_layout()
plt.show()

### 2.2 Autoregressive Language Modeling

**GPT-style подход:**

Предвиждай следващия токен, даден контекст:

$$\mathcal{L}_{AR} = -\sum_{i=1}^{n} \log P(x_i | x_1, x_2, ..., x_{i-1})$$

**Същата формула като n-gram модели от Лекция 1!**

Разликата: невронна мрежа вместо counting statistics.

In [ ]:
# Autoregressive training - всяка позиция е loss
def compute_ar_loss_example(tokens, vocab_probs):
    """
    Демонстрира как се изчислява AR loss.
    vocab_probs: симулирани вероятности за всеки токен.
    """
    total_loss = 0
    print("Autoregressive Loss изчисление:")
    print("="*60)
    
    for i in range(1, len(tokens)):
        context = tokens[:i]
        target = tokens[i]
        prob = vocab_probs.get(target, 0.01)  # Симулирана вероятност
        log_prob = np.log(prob)
        total_loss -= log_prob
        
        print(f"P('{target}' | '{' '.join(context)}') = {prob:.3f}")
        print(f"  -log(P) = {-log_prob:.3f}")
        print()
    
    avg_loss = total_loss / (len(tokens) - 1)
    perplexity = np.exp(avg_loss)
    
    print("="*60)
    print(f"Total Loss: {total_loss:.3f}")
    print(f"Average Loss: {avg_loss:.3f}")
    print(f"Perplexity: {perplexity:.2f}")
    return total_loss, perplexity

# Пример
tokens = ["The", "cat", "sat", "on", "the", "mat"]
# Симулирани вероятности (в реалност идват от модела)
vocab_probs = {"cat": 0.15, "sat": 0.08, "on": 0.20, "the": 0.25, "mat": 0.05}

loss, ppl = compute_ar_loss_example(tokens, vocab_probs)

### Защо Autoregressive победи?

| Фактор | Обяснение |
|--------|----------|
| **Естествена генерация** | Токен по токен, като писане |
| **Без mismatch** | Същият процес при training и inference |
| **Унификация** | Всички задачи като text generation |
| **Scaling** | Емпирично по-добре при мащаб |

In [ ]:
# Просто autoregressive генериране
def simple_ar_generate(start_tokens, vocab, model_fn, max_tokens=10):
    """
    Демонстрира autoregressive генериране.
    model_fn: функция, която дава вероятности за следващ токен.
    """
    tokens = list(start_tokens)
    
    for _ in range(max_tokens):
        # Получаваме вероятности за следващ токен
        probs = model_fn(tokens, vocab)
        
        # Сампълваме следващ токен
        next_token = np.random.choice(vocab, p=probs)
        tokens.append(next_token)
        
        if next_token == '.':
            break
    
    return tokens

# Прост "модел" - uniform с bias към някои думи
def toy_model(context, vocab):
    """Toy model: дава по-висока вероятност на 'cat', 'sat', '.'"""
    probs = np.ones(len(vocab)) * 0.05
    
    # Bias към смислени продължения
    if 'The' in context and 'cat' not in context:
        probs[vocab.index('cat')] = 0.4
    if 'cat' in context and 'sat' not in context:
        probs[vocab.index('sat')] = 0.4
    if 'sat' in context:
        probs[vocab.index('.')] = 0.3
    
    return probs / probs.sum()

vocab = ['The', 'cat', 'sat', 'on', 'mat', 'dog', 'ran', '.']

print("Autoregressive генериране (toy model):")
for i in range(3):
    np.random.seed(42 + i)
    result = simple_ar_generate(['The'], vocab, toy_model, max_tokens=6)
    print(f"  {i+1}. {' '.join(result)}")

### Какво научава моделът при pretraining?

Предвиждането на следващ токен изисква:

| Знание | Пример |
|--------|--------|
| **Граматика** | "The cats **are**" не "The cats **is**" |
| **Факти** | "Paris is the capital of **France**" |
| **Логика** | "If A then B. A is true. Therefore **B**" |
| **Код** | "def factorial(n): **return**" |
| **Стил** | Formal vs informal writing |

In [ ]:
# Прост autoregressive training loop (концептуален)
class TinyLanguageModel(nn.Module):
    """Минимален езиков модел за демонстрация."""
    def __init__(self, vocab_size, d_model=64, n_heads=4, n_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(512, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dim_feedforward=d_model*4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, n_layers)
        self.output = nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        seq_len = x.shape[1]
        positions = torch.arange(seq_len, device=x.device)
        
        # Embeddings + positions
        h = self.embedding(x) + self.pos_embedding(positions)
        
        # Causal mask: -inf за позиции, които не трябва да виждаме
        mask = nn.Transformer.generate_square_subsequent_mask(seq_len, device=x.device)
        
        # Transformer
        h = self.transformer(h, mask=mask, is_causal=True)
        
        # Output logits
        return self.output(h)

# Демонстрация
vocab_size = 1000
model = TinyLanguageModel(vocab_size)
x = torch.randint(0, vocab_size, (2, 10))  # batch=2, seq_len=10

logits = model(x)
print(f"Input shape: {x.shape}")
print(f"Output logits shape: {logits.shape}")
print(f"  (batch_size, seq_len, vocab_size)")
print(f"\nПараметри: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training step демонстрация
def training_step(model, batch, optimizer):
    """
    Един training step за autoregressive LM.
    batch: [batch_size, seq_len] - токенизиран текст
    """
    model.train()
    optimizer.zero_grad()
    
    # Input: всички токени освен последния
    # Target: всички токени освен първия
    input_ids = batch[:, :-1]
    target_ids = batch[:, 1:]
    
    # Forward pass
    logits = model(input_ids)  # [batch, seq_len-1, vocab]
    
    # Loss: cross-entropy за всяка позиция
    loss = F.cross_entropy(
        logits.reshape(-1, logits.size(-1)),  # [batch*seq, vocab]
        target_ids.reshape(-1)                 # [batch*seq]
    )
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    return loss.item()

# Демонстрация на един step
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
batch = torch.randint(0, vocab_size, (4, 32))

loss = training_step(model, batch, optimizer)
perplexity = np.exp(loss)

print(f"Loss: {loss:.4f}")
print(f"Perplexity: {perplexity:.2f}")
print(f"\n(Random init → perplexity ≈ vocab_size = {vocab_size})")

### Perplexity: Основната метрика

$$\text{Perplexity} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N} \log P(x_i | x_{<i})\right)$$

**Интуиция:** Средно колко "объркан" е моделът.

| Perplexity | Значение |
|------------|----------|
| 1 | Перфектно предвиждане |
| vocab_size | Random guessing |
| ~20-30 | Добър LM на английски |

---
## 3. Източници и състав на данните

### Мащаб на данните

| Модел | Токени | Година |
|-------|--------|--------|
| GPT-2 | 40B | 2019 |
| GPT-3 | 300B | 2020 |
| LLaMA | 1.4T | 2023 |
| LLaMA 2 | 2T | 2023 |
| GPT-4 | ~13T (estimate) | 2023 |

In [ ]:
# Визуализация на ръста на training данни
models = ['GPT-2\n(2019)', 'GPT-3\n(2020)', 'Chinchilla\n(2022)', 
          'LLaMA\n(2023)', 'LLaMA 2\n(2023)']
tokens_billions = [40, 300, 1400, 1400, 2000]

fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(models)))

bars = ax.bar(models, tokens_billions, color=colors, edgecolor='black', linewidth=1.2)

# Добавяме стойности върху барове
for bar, tokens in zip(bars, tokens_billions):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 30,
            f'{tokens}B', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('Токени (милиарди)', fontsize=12)
ax.set_title('Експоненциален ръст на training данни', fontsize=14)
ax.set_ylim(0, 2500)

# Добавяме линия на "целия интернет"
ax.axhline(y=2000, color='red', linestyle='--', alpha=0.5)
ax.text(4.5, 2050, '~Лимит на качествен\nтекст в интернета', 
        ha='right', fontsize=10, color='red')

plt.tight_layout()
plt.show()

print("50x ръст за 4 години!")

### Основни източници на данни

#### Web Crawls (60-80% от данните)

| Dataset | Описание | Размер |
|---------|----------|--------|
| **Common Crawl** | Най-голям публичен web crawl | Петабайти HTML |
| **C4** | Filtered Common Crawl | 800GB текст |
| **RefinedWeb** | High-quality filtered | 5T токени |
| **FineWeb** | Още по-качествен | 15T токени |

#### Книги (4-5%)

- **Project Gutenberg:** Public domain класика
- **Books3:** Controversial dataset (legal issues)
- **Качество:** Високо, но ограничен обем

#### Код (4-5%)

- **GitHub:** Огромно repository
- **The Stack:** Curated с лицензи
- **Важно за:** Reasoning, структурирано мислене

#### Научен текст (2-5%)

- **arXiv:** Preprints в science/math
- **PubMed:** Biomedical literature
- **Semantic Scholar:** Academic papers

#### Curated източници (5-10%)

- **Wikipedia:** Висококачествен, factual
- **StackExchange:** Q&A формат
- **Reddit:** Conversational, varied quality

In [ ]:
# Типичен dataset mix (LLaMA-style)
sources = ['Web Crawl', 'Code', 'Wikipedia', 'Books', 'arXiv', 'StackExchange']
percentages = [67, 4.5, 4.5, 4.5, 2.5, 2]
tokens_t = [0.95, 0.065, 0.065, 0.065, 0.035, 0.03]  # В трилиони

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
colors = plt.cm.Set3(np.linspace(0, 1, len(sources)))
explode = [0.05 if p < 5 else 0 for p in percentages]

axes[0].pie(percentages, labels=sources, autopct='%1.1f%%', 
            colors=colors, explode=explode, startangle=90)
axes[0].set_title('Разпределение по източник', fontsize=12)

# Bar chart с токени
bars = axes[1].barh(sources, tokens_t, color=colors, edgecolor='black')
axes[1].set_xlabel('Токени (трилиони)', fontsize=11)
axes[1].set_title('Абсолютен брой токени', fontsize=12)

for bar, t in zip(bars, tokens_t):
    axes[1].text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                 f'{t}T', va='center', fontsize=10)

axes[1].set_xlim(0, 1.1)

plt.tight_layout()
plt.show()

### Защо съставът е важен?

| Проблем | Причина |
|---------|--------|
| Твърде много web | Ниско качество, повторения |
| Твърде много код | Странни генерации |
| Само английски | Лоша multilingual способност |
| Без код | Слаба логика и reasoning |

**Баланс е критичен!**

### Synthetic Data (нов тренд)

**Идея:** Генерирай данни с друг LLM.

**Примери:**
- Math задачи и решения
- Code + тестове
- Instruction-following pairs

**Спорно:** Може ли модел да се учи от себе си?

---
## 4. Качество на данните и филтриране

### Quality vs Quantity

**Ранно вярване:** Повече данни = по-добър модел

**Модерен insight (Chinchilla, Phi):** 
- Качеството е еднакво важно
- Малък модел + качествени данни може да победи голям модел + шумни данни

In [ ]:
# Примери за лошо качество на web data
bad_examples = [
    # Boilerplate
    "Cookie Policy. We use cookies to improve your experience. Accept All | Reject All | Manage Preferences",
    
    # Repetition
    "Buy now! Buy now! Buy now! Best prices! Buy now!",
    
    # Gibberish
    "asdfasdf lorem ipsum dolor sit amet asdfasdf",
    
    # Too short
    "OK",
    
    # Machine generated
    "Product SKU: 12345-ABC | Weight: 2.5kg | Dimensions: 10x20x30cm",
]

good_examples = [
    "The transformer architecture revolutionized natural language processing by enabling parallel computation and capturing long-range dependencies.",
    "Machine learning models learn patterns from data. The key is to have representative training examples.",
]

print("Примери за ЛОШО качество (трябва да се филтрира):")
print("=" * 60)
for i, ex in enumerate(bad_examples, 1):
    print(f"{i}. {ex[:70]}..." if len(ex) > 70 else f"{i}. {ex}")
    
print("\nПримери за ДОБРО качество:")
print("=" * 60)
for i, ex in enumerate(good_examples, 1):
    print(f"{i}. {ex}")

### Heuristic Filters

Прости правила за филтриране:

| Филтър | Описание | Threshold |
|--------|----------|----------|
| Дължина | Премахни много кратки документи | < 50 думи |
| Повторения | Много еднакви думи/n-grams | > 20% repeats |
| Специални символи | Твърде много punct/digits | > 30% |
| Дължина на думи | Средна дължина | < 3 или > 10 |

In [ ]:
# Имплементация на heuristic filters
def quality_filters(text):
    """Връща score от 0 до 1 и причини за low score."""
    issues = []
    words = text.split()
    
    # 1. Length filter
    if len(words) < 10:
        issues.append(f"Too short ({len(words)} words)")
    
    # 2. Repetition filter
    if len(words) > 0:
        word_counts = Counter(words)
        most_common_count = word_counts.most_common(1)[0][1]
        repetition_ratio = most_common_count / len(words)
        if repetition_ratio > 0.2:
            issues.append(f"High repetition ({repetition_ratio:.1%})")
    
    # 3. Special character ratio
    if len(text) > 0:
        special_chars = sum(1 for c in text if not c.isalnum() and c != ' ')
        special_ratio = special_chars / len(text)
        if special_ratio > 0.3:
            issues.append(f"Too many special chars ({special_ratio:.1%})")
    
    # 4. Average word length
    if len(words) > 0:
        avg_word_len = np.mean([len(w) for w in words])
        if avg_word_len < 2 or avg_word_len > 15:
            issues.append(f"Unusual word length ({avg_word_len:.1f})")
    
    # Score: 1 if no issues, lower otherwise
    score = max(0, 1 - len(issues) * 0.25)
    
    return score, issues

# Тестваме
test_texts = bad_examples + good_examples

print("Quality Filter Results:")
print("=" * 70)
for text in test_texts:
    score, issues = quality_filters(text)
    status = "PASS" if score >= 0.75 else "FAIL"
    print(f"[{status}] Score: {score:.2f}")
    print(f"    Text: {text[:50]}...")
    if issues:
        print(f"    Issues: {', '.join(issues)}")
    print()

### Classifier-Based Filtering

**Идея:** Обучи класификатор на "високо качество" vs "ниско качество"

**Положителни примери:** Wikipedia, учебници, научни статии

**Отрицателни примери:** Random web pages

**Използва се от:** RefinedWeb, FineWeb, GPT-4 data pipeline

### Perplexity Filtering

**Идея:** Използвай reference модел да оцени текст.

- **Много ниска perplexity:** Прекалено просто/повтарящо се
- **Много висока perplexity:** Gibberish или грешен език
- **Средна perplexity:** Оптимално!

```
Keep if: threshold_low < PPL < threshold_high
```

### 4.1 Deduplication: Защо е критично

**Проблемите с дупликати:**

| Проблем | Описание |
|---------|----------|
| Пропиляна compute | Учим едно и също много пъти |
| Меморизация | Моделът запомня verbatim |
| Benchmark leak | Тестовете може да са в training |
| Biased representations | Overrepresented content |

In [ ]:
# Мащаб на проблема с дупликати
datasets = ['Raw\nCommon Crawl', 'C4', 'RefinedWeb']
duplicate_rates = [45, 25, 8]  # Процент дупликати

colors = ['#ff6b6b', '#ffa502', '#2ed573']
fig, ax = plt.subplots(figsize=(8, 5))

bars = ax.bar(datasets, duplicate_rates, color=colors, edgecolor='black', linewidth=1.5)

for bar, rate in zip(bars, duplicate_rates):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate}%', ha='center', fontsize=12, fontweight='bold')

ax.set_ylabel('Процент near-duplicates', fontsize=11)
ax.set_title('Дупликати в различни datasets\n(преди/след филтриране)', fontsize=12)
ax.set_ylim(0, 55)

# Анотация
ax.annotate('Почти половината!', xy=(0, 45), xytext=(0.5, 50),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red')

plt.tight_layout()
plt.show()

### Exact Deduplication

**Просто:** Hash документ → премахни еднакви hash-ове.

```python
doc_hash = hashlib.md5(doc.encode()).hexdigest()
if doc_hash in seen_hashes:
    skip(doc)
```

**Недостатък:** Не хваща "почти еднакви" документи.

In [ ]:
# Exact deduplication demo
def exact_dedup(documents):
    """Премахва точни дупликати чрез hashing."""
    seen = set()
    unique = []
    duplicates = 0
    
    for doc in documents:
        doc_hash = hashlib.md5(doc.encode()).hexdigest()
        if doc_hash not in seen:
            seen.add(doc_hash)
            unique.append(doc)
        else:
            duplicates += 1
    
    return unique, duplicates

# Тест
docs = [
    "The cat sat on the mat.",
    "The dog ran in the park.",
    "The cat sat on the mat.",  # Exact duplicate
    "The cat sat on the mat!",  # Near-duplicate (different punctuation)
    "The dog ran in the park.",  # Exact duplicate
]

unique, dups = exact_dedup(docs)
print(f"Original: {len(docs)} documents")
print(f"After exact dedup: {len(unique)} unique, {dups} duplicates removed")
print(f"\nUnique documents:")
for doc in unique:
    print(f"  - {doc}")

print("\n⚠️ Near-duplicate 'mat!' не е хванат!")

### MinHash: Near-Duplicate Detection

**Идея:** Апроксимира Jaccard similarity между документи.

$$\text{Jaccard}(A, B) = \frac{|A \cap B|}{|A \cup B|}$$

**Алгоритъм:**
1. Раздели документ на n-grams (shingles)
2. Приложи k различни hash функции
3. За всяка функция запази минималния hash
4. Signature = [min_hash_1, min_hash_2, ..., min_hash_k]

**Ключово:** $P(\text{min\_hash равни}) = \text{Jaccard}(A, B)$

In [ ]:
# MinHash имплементация от scratch
class MinHash:
    def __init__(self, num_hashes=100):
        self.num_hashes = num_hashes
        # Генерираме random hash параметри
        self.a = np.random.randint(1, 2**31, num_hashes)
        self.b = np.random.randint(0, 2**31, num_hashes)
        self.prime = 2**31 - 1  # Mersenne prime
    
    def _hash(self, x, i):
        """Hash функция i за стойност x."""
        return (self.a[i] * x + self.b[i]) % self.prime
    
    def get_shingles(self, text, k=3):
        """Генерира k-grams (shingles) от текст."""
        words = text.lower().split()
        shingles = set()
        for i in range(len(words) - k + 1):
            shingle = " ".join(words[i:i+k])
            shingles.add(hash(shingle))
        return shingles
    
    def signature(self, text):
        """Изчислява MinHash signature за текст."""
        shingles = self.get_shingles(text)
        sig = np.full(self.num_hashes, np.inf)
        
        for shingle in shingles:
            for i in range(self.num_hashes):
                hash_val = self._hash(shingle, i)
                sig[i] = min(sig[i], hash_val)
        
        return sig
    
    def similarity(self, sig1, sig2):
        """Jaccard approximation от signatures."""
        return np.mean(sig1 == sig2)

# Демонстрация
minhash = MinHash(num_hashes=128)

doc1 = "The cat sat on the mat in the living room"
doc2 = "The cat sat on the mat in the kitchen"  # Near-duplicate
doc3 = "Dogs are wonderful pets that bring joy"  # Different

sig1 = minhash.signature(doc1)
sig2 = minhash.signature(doc2)
sig3 = minhash.signature(doc3)

print("MinHash Similarity (Jaccard approximation):")
print(f"  doc1 vs doc2: {minhash.similarity(sig1, sig2):.3f} (near-duplicates)")
print(f"  doc1 vs doc3: {minhash.similarity(sig1, sig3):.3f} (different)")
print(f"  doc2 vs doc3: {minhash.similarity(sig2, sig3):.3f} (different)")

In [ ]:
# True Jaccard за сравнение
def true_jaccard(text1, text2, k=3):
    """Изчислява истинската Jaccard similarity."""
    words1 = text1.lower().split()
    words2 = text2.lower().split()
    
    shingles1 = set(" ".join(words1[i:i+k]) for i in range(len(words1)-k+1))
    shingles2 = set(" ".join(words2[i:i+k]) for i in range(len(words2)-k+1))
    
    intersection = len(shingles1 & shingles2)
    union = len(shingles1 | shingles2)
    
    return intersection / union if union > 0 else 0

print("Сравнение: True Jaccard vs MinHash approximation")
print("="*50)
pairs = [(doc1, doc2, "near-dup"), (doc1, doc3, "different"), (doc2, doc3, "different")]

for d1, d2, label in pairs:
    true_j = true_jaccard(d1, d2)
    mh_j = minhash.similarity(minhash.signature(d1), minhash.signature(d2))
    print(f"{label:12} True: {true_j:.3f}  MinHash: {mh_j:.3f}  Error: {abs(true_j-mh_j):.3f}")

In [ ]:
# Визуализация: MinHash accuracy vs брой hash функции
num_hashes_range = [8, 16, 32, 64, 128, 256, 512]
errors = []

true_sim = true_jaccard(doc1, doc2)

for n_hash in num_hashes_range:
    # Average over multiple trials
    trial_errors = []
    for _ in range(20):
        mh = MinHash(num_hashes=n_hash)
        est_sim = mh.similarity(mh.signature(doc1), mh.signature(doc2))
        trial_errors.append(abs(true_sim - est_sim))
    errors.append(np.mean(trial_errors))

plt.figure(figsize=(10, 5))
plt.plot(num_hashes_range, errors, 'bo-', linewidth=2, markersize=8)
plt.xscale('log', base=2)
plt.xlabel('Брой hash функции', fontsize=11)
plt.ylabel('Средна грешка в Jaccard', fontsize=11)
plt.title('MinHash: Повече hashes → По-точна оценка', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(num_hashes_range, [str(n) for n in num_hashes_range])

# Annotation
plt.annotate('128 hashes е\nдобър баланс', xy=(128, errors[4]), 
             xytext=(200, errors[4]+0.03),
             arrowprops=dict(arrowstyle='->', color='green'),
             fontsize=10, color='green')

plt.tight_layout()
plt.show()

### LSH Banding за ефективно търсене

**Проблем:** O(n²) сравнения между всички документи е твърде бавно.

**Решение:** Locality-Sensitive Hashing (LSH)

1. Раздели signature на bands
2. Hash всеки band в bucket
3. Само документи в същия bucket се сравняват

**Резултат:** От O(n²) към почти O(n)

---
## 5. Контаминация и evaluation integrity

### Проблемът с контаминацията

**Контаминация:** Test данни попадат в training set.

**Резултат:** 
- Inflated benchmark scores
- Моделът "знае" отговорите
- Невалидна evaluation

In [ ]:
# Пример: Контаминация
benchmark_example = {
    "question": "What is the capital of France?",
    "answer": "Paris"
}

training_documents = [
    "France is a country in Western Europe. Its capital is Paris.",  # OK - factual
    "The capital of France is Paris, which is also its largest city.",  # OK - factual
    "Q: What is the capital of France? A: Paris",  # CONTAMINATION!
    "Test question: What is the capital of France? Correct answer: Paris",  # CONTAMINATION!
]

print("Benchmark:")
print(f"  Q: {benchmark_example['question']}")
print(f"  A: {benchmark_example['answer']}")

print("\nTraining documents:")
for i, doc in enumerate(training_documents, 1):
    is_contaminated = "Q:" in doc or "Test question" in doc
    status = "❌ CONTAMINATED" if is_contaminated else "✓ OK"
    print(f"  {i}. [{status}] {doc}")

### Типове контаминация

| Тип | Описание | Severity |
|-----|----------|----------|
| **Direct** | Точен test example в training | Критичен |
| **Indirect** | Парафраза на test example | Висок |
| **Temporal** | Training data от след benchmark | Среден |
| **Partial** | Само въпрос или само отговор | Нисък |

### Detection methods

#### N-gram Overlap

Проверяваме дали n-grams от test set се срещат в training.

In [ ]:
def ngram_overlap(text1, text2, n=8):
    """
    Изчислява n-gram overlap между два текста.
    Висок overlap = potential contamination.
    """
    def get_ngrams(text, n):
        words = text.lower().split()
        return set(" ".join(words[i:i+n]) for i in range(len(words)-n+1))
    
    ngrams1 = get_ngrams(text1, n)
    ngrams2 = get_ngrams(text2, n)
    
    if not ngrams1 or not ngrams2:
        return 0
    
    overlap = len(ngrams1 & ngrams2)
    return overlap / min(len(ngrams1), len(ngrams2))

# Test
test_question = "What is the capital of France and what is its population?"

training_docs = [
    "France is known for its culture. Paris has many museums.",  # Low overlap
    "The capital of France is Paris. Its population is about 2 million.",  # Medium overlap
    "Question: What is the capital of France and what is its population?",  # High overlap!
]

print(f"Test: '{test_question}'\n")
print("N-gram overlap (n=5) с training documents:")
for doc in training_docs:
    overlap = ngram_overlap(test_question, doc, n=5)
    status = "⚠️ POTENTIAL CONTAMINATION" if overlap > 0.3 else "OK"
    print(f"  {overlap:.2%} - {status}")
    print(f"    Doc: {doc[:60]}...")

### Canary Strings

**Идея:** Вмъкни уникални "canary" strings в training data.

```
The secret canary code is: X7K9M2P4
```

Ако моделът може да възпроизведе canary → memorization риск.

### Mitigation Strategies

| Стратегия | Описание |
|-----------|----------|
| **Decontamination** | Премахни test-like примери от training |
| **Temporal splits** | Training само от преди benchmark |
| **New benchmarks** | Създавай нови тестове редовно |
| **Private test sets** | Не публикувай test данни |

In [ ]:
# Простa decontamination функция
def decontaminate(training_docs, test_examples, threshold=0.3, n=8):
    """
    Премахва training документи с високо overlap с test.
    """
    clean_docs = []
    removed = 0
    
    for doc in training_docs:
        is_contaminated = False
        for test in test_examples:
            if ngram_overlap(doc, test, n) > threshold:
                is_contaminated = True
                break
        
        if is_contaminated:
            removed += 1
        else:
            clean_docs.append(doc)
    
    return clean_docs, removed

# Demo
test_set = [
    "What is the capital of France?",
    "Who wrote Romeo and Juliet?",
]

training_set = [
    "France is a European country.",
    "Q: What is the capital of France? A: Paris",  # Contaminated
    "Shakespeare was born in Stratford.",
    "Test: Who wrote Romeo and Juliet? Answer: Shakespeare",  # Contaminated
    "The weather in Paris is mild.",
]

clean, removed = decontaminate(training_set, test_set, threshold=0.2, n=4)
print(f"Original training: {len(training_set)} docs")
print(f"After decontamination: {len(clean)} docs ({removed} removed)")
print(f"\nClean documents:")
for doc in clean:
    print(f"  - {doc}")

---
## 6. Scaling Laws

### Емпиричното откритие

**OpenAI (Kaplan et al., 2020):** Loss се предвижда от compute!

- Забележително consistent
- Работи през много порядъци на величина
- Power law relationship

### Трите оси на scaling

| Ос | Символ | Описание |
|----|--------|----------|
| **Model size** | N | Брой параметри |
| **Data size** | D | Брой токени |
| **Compute** | C | FLOPs за training |

### Scaling Law формули

$$L(N) = \left(\frac{N_c}{N}\right)^{\alpha_N}$$

$$L(D) = \left(\frac{D_c}{D}\right)^{\alpha_D}$$

$$L(C) = \left(\frac{C_c}{C}\right)^{\alpha_C}$$

Типични експоненти: $\alpha \approx 0.05 - 0.1$

**Интерпретация:** 10x повече compute → ~0.1-0.2 намаление в loss

In [ ]:
# Симулация на scaling laws
def scaling_law(x, x_c, alpha):
    """Power law: L = (x_c/x)^alpha"""
    return (x_c / x) ** alpha

# Параметри (приблизителни от литературата)
alpha_N = 0.076  # Model size exponent
alpha_D = 0.095  # Data size exponent
N_c = 8.8e13     # Critical model size
D_c = 5.4e13     # Critical data size

# Ranges
model_sizes = np.logspace(6, 12, 50)  # 1M to 1T params
data_sizes = np.logspace(9, 13, 50)   # 1B to 10T tokens

# Loss curves
loss_N = scaling_law(model_sizes, N_c, alpha_N) + 1.5  # +baseline
loss_D = scaling_law(data_sizes, D_c, alpha_D) + 1.5

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss vs Model Size
ax = axes[0]
ax.loglog(model_sizes, loss_N, 'b-', linewidth=2)
ax.set_xlabel('Model Parameters', fontsize=11)
ax.set_ylabel('Test Loss', fontsize=11)
ax.set_title('Scaling Law: Loss vs Model Size', fontsize=12)
ax.grid(True, alpha=0.3)

# Маркери за известни модели
known_models = {
    'GPT-2': 1.5e9,
    'GPT-3': 175e9,
    'LLaMA 70B': 70e9,
}
for name, size in known_models.items():
    loss = scaling_law(size, N_c, alpha_N) + 1.5
    ax.scatter([size], [loss], s=100, zorder=5)
    ax.annotate(name, (size, loss), xytext=(5, 5), textcoords='offset points', fontsize=9)

# Loss vs Data Size
ax = axes[1]
ax.loglog(data_sizes, loss_D, 'r-', linewidth=2)
ax.set_xlabel('Training Tokens', fontsize=11)
ax.set_ylabel('Test Loss', fontsize=11)
ax.set_title('Scaling Law: Loss vs Data Size', fontsize=12)
ax.grid(True, alpha=0.3)

# Маркери за известни training runs
known_data = {
    'GPT-3': 300e9,
    'LLaMA': 1.4e12,
    'LLaMA 2': 2e12,
}
for name, tokens in known_data.items():
    loss = scaling_law(tokens, D_c, alpha_D) + 1.5
    ax.scatter([tokens], [loss], s=100, zorder=5)
    ax.annotate(name, (tokens, loss), xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

### Chinchilla: Compute-Optimal Training

**DeepMind (Hoffmann et al., 2022):** Моделите са били undertrained!

**Ключов insight:**

$$\text{Optimal ratio} \approx 20 \text{ tokens per parameter}$$

| Модел | Parameters | Tokens | Ratio |
|-------|------------|--------|-------|
| GPT-3 | 175B | 300B | 1.7 (undertrained) |
| Chinchilla | 70B | 1.4T | 20 (optimal) |
| LLaMA | 7B | 1.4T | 200 (overtrained) |

In [ ]:
# Chinchilla optimal frontier
def chinchilla_optimal(compute_budget, a=0.5, b=0.5):
    """
    Изчислява оптимален model size и data size за даден compute budget.
    Опростена версия: N ~ C^a, D ~ C^b где a + b = 1
    """
    N_opt = compute_budget ** a
    D_opt = compute_budget ** b
    return N_opt, D_opt

# Различни compute budgets
compute_budgets = np.logspace(18, 24, 20)  # FLOPs

# За фиксиран compute: tradeoff между N и D
fig, ax = plt.subplots(figsize=(10, 6))

C_fixed = 1e21  # Fixed compute budget
# Constraint: N * D ≈ C (опростено)
N_range = np.logspace(9, 12, 100)
D_range = C_fixed / N_range

# Loss за различни combinations (опростен модел)
def combined_loss(N, D, alpha_N=0.076, alpha_D=0.095):
    return (N_c/N)**alpha_N + (D_c/D)**alpha_D + 1.5

losses = [combined_loss(n, d) for n, d in zip(N_range, D_range)]

ax.semilogx(N_range, losses, 'b-', linewidth=2)
ax.set_xlabel('Model Parameters (N)', fontsize=11)
ax.set_ylabel('Loss', fontsize=11)
ax.set_title(f'Loss vs Model Size при фиксиран compute (C={C_fixed:.0e} FLOPs)', fontsize=12)

# Optimal point
optimal_idx = np.argmin(losses)
ax.scatter([N_range[optimal_idx]], [losses[optimal_idx]], s=150, c='red', zorder=5)
ax.annotate(f'Optimal\nN={N_range[optimal_idx]:.1e}', 
            (N_range[optimal_idx], losses[optimal_idx]),
            xytext=(20, 20), textcoords='offset points',
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red')

# Показваме GPT-3 style (голям модел, малко данни) vs Chinchilla style
ax.axvline(x=175e9, color='gray', linestyle='--', alpha=0.5, label='GPT-3 size')
ax.axvline(x=70e9, color='green', linestyle='--', alpha=0.5, label='Chinchilla size')
ax.legend()

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"При compute budget {C_fixed:.0e} FLOPs:")
print(f"  Optimal model size: {N_range[optimal_idx]:.2e} параметри")
print(f"  Optimal data size: {D_range[optimal_idx]:.2e} токени")
print(f"  Ratio: {D_range[optimal_idx]/N_range[optimal_idx]:.1f} tokens/param")

### Какво scale-ва добре и какво не

| Scale-ва добре | Scale-ва непредвидимо |
|----------------|----------------------|
| Perplexity | Reasoning tasks |
| Повечето NLP tasks | Specific capabilities |
| Factual knowledge | Alignment/safety |
| Code generation | Emergent abilities |

### Кога scaling се чупи

| Лимит | Описание |
|-------|----------|
| **Data exhaustion** | Свършва качественият текст |
| **Compute** | Твърде скъпо/бавно |
| **Diminishing returns** | Подобренията намаляват |
| **Capability gaps** | Някои неща не се учат със scale |

---
## 7. Обучение в мащаб

### Computational Requirements

| Модел | GPU Hours | Estimated Cost |
|-------|-----------|----------------|
| GPT-2 (1.5B) | ~5K A100 | ~$50K |
| GPT-3 (175B) | ~3.6M A100 | ~$5-10M |
| LLaMA 65B | ~1M A100 | ~$2-5M |
| GPT-4 | Unknown | ~$50-100M (estimate) |

In [ ]:
# Compute estimation
def estimate_training_flops(params, tokens):
    """
    Приблизителна оценка на FLOPs за training.
    Rule of thumb: 6 * N * D FLOPs (forward + backward)
    """
    return 6 * params * tokens

def flops_to_gpu_hours(flops, gpu_tflops=312):
    """
    Конвертира FLOPs към GPU hours.
    A100: ~312 TFLOPS (FP16)
    """
    gpu_flops_per_hour = gpu_tflops * 1e12 * 3600
    # Assume 50% utilization
    effective_flops = gpu_flops_per_hour * 0.5
    return flops / effective_flops

# Примери
models = [
    ('GPT-2', 1.5e9, 40e9),
    ('LLaMA 7B', 7e9, 1.4e12),
    ('LLaMA 65B', 65e9, 1.4e12),
    ('GPT-3', 175e9, 300e9),
]

print("Training Compute Estimates:")
print("=" * 70)
print(f"{'Model':<15} {'Params':<12} {'Tokens':<12} {'FLOPs':<15} {'GPU Hours':<12}")
print("-" * 70)

for name, params, tokens in models:
    flops = estimate_training_flops(params, tokens)
    gpu_hours = flops_to_gpu_hours(flops)
    
    print(f"{name:<15} {params:.1e}  {tokens:.1e}  {flops:.2e}  {gpu_hours:,.0f}")

print("\n* При 50% GPU utilization на A100 (312 TFLOPS FP16)")

### Distributed Training Strategies

| Strategy | Описание | Кога се използва |
|----------|----------|------------------|
| **Data Parallel** | Копие на модела на всеки GPU | Малки модели |
| **Tensor Parallel** | Разделяме layers между GPUs | Много големи layers |
| **Pipeline Parallel** | Различни layers на различни GPUs | Много дълбоки модели |
| **FSDP** | Sharded параметри и оптимизатор | Най-ефективно за LLMs |

In [ ]:
# Визуализация на parallelism strategies
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Data Parallel
ax = axes[0]
for i in range(4):
    ax.add_patch(plt.Rectangle((i*2.5, 0), 2, 3, fill=True, 
                                color='steelblue', alpha=0.7))
    ax.text(i*2.5 + 1, 1.5, f'Model\ncopy {i+1}', ha='center', va='center',
            fontsize=9, color='white')
    ax.text(i*2.5 + 1, -0.5, f'Batch {i+1}', ha='center', fontsize=9)
ax.set_xlim(-0.5, 10.5)
ax.set_ylim(-1, 4)
ax.axis('off')
ax.set_title('Data Parallel\n(същият модел, различни данни)', fontsize=11)

# Tensor Parallel
ax = axes[1]
colors = plt.cm.Set2(np.linspace(0, 1, 4))
for i in range(4):
    ax.add_patch(plt.Rectangle((i*2.5, 0), 2, 3, fill=True,
                                color=colors[i], alpha=0.8))
    ax.text(i*2.5 + 1, 1.5, f'Layer\npart {i+1}', ha='center', va='center',
            fontsize=9)
ax.set_xlim(-0.5, 10.5)
ax.set_ylim(-1, 4)
ax.axis('off')
ax.set_title('Tensor Parallel\n(един layer разделен)', fontsize=11)

# Pipeline Parallel
ax = axes[2]
for i in range(4):
    ax.add_patch(plt.Rectangle((2, i*0.8), 6, 0.7, fill=True,
                                color=colors[i], alpha=0.8))
    ax.text(5, i*0.8 + 0.35, f'Layers {i*8+1}-{(i+1)*8}', ha='center', va='center',
            fontsize=9)
    ax.text(0.5, i*0.8 + 0.35, f'GPU {i+1}', ha='center', va='center', fontsize=9)
# Arrows
for i in range(3):
    ax.annotate('', xy=(5, (i+1)*0.8), xytext=(5, i*0.8+0.7),
                arrowprops=dict(arrowstyle='->', color='black'))
ax.set_xlim(-0.5, 10.5)
ax.set_ylim(-0.5, 4)
ax.axis('off')
ax.set_title('Pipeline Parallel\n(различни layers на различни GPUs)', fontsize=11)

plt.tight_layout()
plt.show()

### Key Hyperparameters

| Parameter | Typical Value | Notes |
|-----------|---------------|-------|
| Learning rate | 1e-4 to 6e-4 | Warmup + cosine decay |
| Batch size | 2M-4M tokens | Gradient accumulation |
| Weight decay | 0.1 | AdamW |
| Warmup steps | 2000 | Linear warmup |
| Gradient clipping | 1.0 | Stability |

In [ ]:
# Learning rate schedule visualization
def lr_schedule(step, warmup_steps=2000, max_steps=100000, 
                max_lr=3e-4, min_lr=3e-5):
    """Warmup + cosine decay schedule."""
    if step < warmup_steps:
        # Linear warmup
        return max_lr * step / warmup_steps
    else:
        # Cosine decay
        progress = (step - warmup_steps) / (max_steps - warmup_steps)
        return min_lr + 0.5 * (max_lr - min_lr) * (1 + np.cos(np.pi * progress))

steps = np.arange(0, 100000, 100)
lrs = [lr_schedule(s) for s in steps]

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Full schedule
ax = axes[0]
ax.plot(steps, lrs, 'b-', linewidth=2)
ax.axvline(x=2000, color='red', linestyle='--', alpha=0.5, label='Warmup end')
ax.set_xlabel('Training Step', fontsize=11)
ax.set_ylabel('Learning Rate', fontsize=11)
ax.set_title('Learning Rate Schedule: Warmup + Cosine Decay', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# Zoom on warmup
ax = axes[1]
warmup_steps = np.arange(0, 3000, 10)
warmup_lrs = [lr_schedule(s) for s in warmup_steps]
ax.plot(warmup_steps, warmup_lrs, 'b-', linewidth=2)
ax.axvline(x=2000, color='red', linestyle='--', alpha=0.5, label='Warmup end')
ax.set_xlabel('Training Step', fontsize=11)
ax.set_ylabel('Learning Rate', fontsize=11)
ax.set_title('Warmup Phase (zoom)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Training Failures

| Problem | Cause | Solution |
|---------|-------|----------|
| **Loss spikes** | Bad batch, LR too high | Lower LR, skip batch |
| **NaN loss** | Numerical instability | Gradient clipping, FP32 |
| **No convergence** | LR too low/high | Tune LR |
| **Hardware failure** | GPU dies | Checkpoints |

In [ ]:
# Симулация на training loss curve с проблеми
np.random.seed(42)
steps = np.arange(0, 50000, 100)

# Base loss curve (smooth decay)
base_loss = 4.0 * np.exp(-steps/20000) + 1.5 + np.random.randn(len(steps)) * 0.02

# Добавяме "spikes"
spike_steps = [10000, 25000, 35000]
for spike_step in spike_steps:
    idx = np.abs(steps - spike_step).argmin()
    base_loss[idx:idx+3] += np.array([0.5, 1.0, 0.3])

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(steps, base_loss, 'b-', linewidth=1.5, alpha=0.8)

# Mark spikes
for spike_step in spike_steps:
    ax.axvline(x=spike_step, color='red', linestyle='--', alpha=0.3)
    ax.text(spike_step, 5.5, 'spike', ha='center', fontsize=9, color='red')

ax.set_xlabel('Training Step', fontsize=11)
ax.set_ylabel('Loss', fontsize=11)
ax.set_title('Typical Training Loss Curve\n(с loss spikes)', fontsize=12)
ax.set_ylim(1, 6)
ax.grid(True, alpha=0.3)

# Annotation
ax.annotate('Loss spikes: нормално\nпри large-scale training', 
            xy=(25000, 3), xytext=(35000, 4),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

---
## 8. От base model към useful model

### Base Model Limitations

**Base model** (след pretraining) може да:
- Предвижда следващ токен много добре
- Генерира coherent text

**НО не е "полезен":**
- Не следва инструкции
- Не е безопасен
- Непредвидимо поведение

In [ ]:
# Демонстрация: Base model vs Assistant
user_prompt = "What is the capital of France?"

base_model_completions = [
    "What is the capital of France? What is the capital of Germany? What is the capital of Italy?",
    "What is the capital of France? A. Paris B. London C. Berlin D. Madrid",
    "What is the capital of France? I'm writing a geography quiz and need...",
]

assistant_completion = "The capital of France is Paris."

print("User: What is the capital of France?")
print("\n" + "="*60)
print("BASE MODEL (просто предвижда следващ токен):")
for i, comp in enumerate(base_model_completions, 1):
    print(f"  {i}. {comp}")

print("\n" + "="*60)
print("ASSISTANT MODEL (instruction-tuned):")
print(f"  {assistant_completion}")

print("\n→ Base model предвижда какво би следвало в training data")
print("→ Assistant model е обучен да ОТГОВАРЯ на въпроси")

### Модерният Pipeline

```
Pretraining → SFT → RLHF → Deploy
(тази лекция)   (Лекция 8)
```

| Етап | Цел | Данни |
|------|-----|-------|
| **Pretraining** | Научи езика | Трилиони токени от интернета |
| **SFT** | Научи да следва инструкции | ~100K instruction pairs |
| **RLHF** | Align с човешки preferences | ~50K comparisons |

### Preview: Instruction Tuning (SFT)

**Supervised Fine-Tuning:**

Training data format:
```
User: <instruction>
Assistant: <response>
```

**Ефект:** Моделът научава да отговаря вместо да продължава.

### Preview: RLHF

**Reinforcement Learning from Human Feedback:**

1. Генерирай множество отговори
2. Хора ги ранкират
3. Обучи reward model
4. Fine-tune с RL

**Детайли в Лекция 8!**

---
## 9. Обобщение

### Ключови изводи

1. **Autoregressive pretraining** победи: предвиждай следващия токен в мащаб

2. **Качеството на данните** е критично: филтриране, дедупликация

3. **Състав на данните** влияе на capabilities: web + code + books + wiki

4. **Scaling laws** предвиждат performance: loss ~ (compute)^(-α)

5. **Chinchilla insight:** balance между model size и data

6. **Контаминация** е реален проблем: detection и decontamination

7. **Base models** не са директно полезни: нужни SFT и RLHF

### Следваща лекция: Emergent Capabilities

**Лекция 7** разглежда:

- Какви способности се появяват при scale?
- Zero-shot и few-shot learning
- In-context learning механика
- Reasoning emergence
- Когато scale не е достатъчен

---
## Ресурси

### Основни статии

1. **"Language Models are Few-Shot Learners"** — Brown et al. (2020) — GPT-3 paper
2. **"Scaling Laws for Neural Language Models"** — Kaplan et al. (2020) — Scaling laws
3. **"Training Compute-Optimal Large Language Models"** — Hoffmann et al. (2022) — Chinchilla
4. **"LLaMA: Open and Efficient Foundation Language Models"** — Touvron et al. (2023)

### Данни и качество

1. **"The Pile: An 800GB Dataset of Diverse Text"** — Gao et al. (2020)
2. **"Deduplicating Training Data Makes Language Models Better"** — Lee et al. (2021)
3. **"The RefinedWeb Dataset for Falcon LLM"** — Penedo et al. (2023)

### Online ресурси

1. **Hugging Face Datasets** — datasets catalog
2. **Common Crawl** — commoncrawl.org
3. **Stanford CS336** — Building LLMs course

---
## Упражнения

### Упражнение 1: Autoregressive Training
- Обучете малък LM на детски разкази
- Измерете perplexity
- Генерирайте текст и оценете качеството

### Упражнение 2: Data Quality Filtering
- Вземете sample от Common Crawl
- Имплементирайте heuristic филтри
- Измерете какъв % преминава

### Упражнение 3: MinHash Deduplication
- Имплементирайте MinHash + LSH
- Намерете near-duplicates в dataset
- Анализирайте duplicate rate

### Упражнение 4: Scaling Law Analysis
- Обучете модели с различен размер
- Фитнете power law
- Предвидете performance за по-голям модел

### Упражнение 5: Contamination Detection
- Проверете dataset за benchmark overlap
- Имплементирайте n-gram detection
- Оценете severity на контаминацията

---
## Край на Лекция 6

**Въпроси?**

---

**Следваща лекция:** Emergent Capabilities at Scale